## - **Importando Bibliotecas**

In [7]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## - **Entrando no Indeed**

In [8]:
driver = webdriver.Chrome()
url_login = "https://br.indeed.com/"
driver.get(url_login)

## - **Pequisando Vagas**

In [9]:
driver.get(url_login)

try:
    recusar_cookies = driver.find_element(By.ID, "onetrust-reject-all-handler")
    recusar_cookies.click()
except:
    pass


input_cargo = "Desenvolvedor Python"
input_localidade = "Minas Gerais"


localidade = driver.find_element(By.ID, "text-input-where")
cargo = driver.find_element(By.ID, "text-input-what")


cargo.send_keys(Keys.CONTROL + "a")
cargo.send_keys(Keys.BACKSPACE)

sleep(1.5)

localidade.send_keys(Keys.CONTROL + "a")
localidade.send_keys(Keys.BACKSPACE)

sleep(1.5)

cargo.send_keys(input_cargo)
localidade.send_keys(input_localidade)
localidade.send_keys(Keys.ENTER)

## - **Iterando sobre Vagas**

In [10]:
ul_element = driver.find_element(By.CSS_SELECTOR, ".css-zu9cdh.eu4oa1w0")

vagas = ul_element.find_elements(By.CLASS_NAME, "css-5lfssm.eu4oa1w0")

print(len(vagas))

for i, vaga in enumerate(vagas):
    if i == 5:
        continue

    if i == 11:
        continue

    if i == 17:
        continue

    print(i)
    vaga.click()
    sleep(2)


18
0
1
2
3
4
6
7
8
9
10
12
13
14
15
16
